In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import pyplot as plt
# from sklearn.datasets import load_breast_cancer
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import KNNImputer

In [45]:
df = pd.read_csv('NY property data.csv')

# Step 1: Filling Missing Value

In [46]:
# check all the missing values
print(df.isnull().sum())

RECORD            0
BBLE              0
B                 0
BLOCK             0
LOT               0
EASEMENT    1066358
OWNER         31745
BLDGCL            0
TAXCLASS          0
LTFRONT           0
LTDEPTH           0
EXT          716689
STORIES       56264
FULLVAL           0
AVLAND            0
AVTOT             0
EXLAND            0
EXTOT             0
EXCD1        432506
STADDR          676
ZIP           29890
EXMPTCL     1055415
BLDFRONT          0
BLDDEPTH          0
AVLAND2      788268
AVTOT2       788262
EXLAND2      983545
EXTOT2       940166
EXCD2        978046
PERIOD            0
YEAR              0
VALTYPE           0
dtype: int64


## Filling ZIP & STORIES

In [47]:
# fill in missing value of ZIP
df['ZIP']=df.groupby(['B'])['ZIP'].transform(lambda x: x.fillna(x.mode()[0]))
# convert ZIP from numerical to string
df.loc[:,'ZIP'] = [str(i)[:5] for i in df['ZIP']]

In [48]:
df['STORIES'].isnull()

0           True
1           True
2          False
3          False
4          False
           ...  
1070989    False
1070990    False
1070991    False
1070992    False
1070993    False
Name: STORIES, Length: 1070994, dtype: bool

In [49]:
# fill in missing value of STORIES
df['STORIES'] = df.groupby(['B','TAXCLASS','ZIP'])['STORIES'].transform(lambda x: x.fillna(x.median()) if x.notnull().sum()>=5 else x)
df['STORIES'] = df.groupby(['B','TAXCLASS'])['STORIES'].transform(lambda x: x.fillna(x.median()) if x.notnull().sum()>=5 else x)
df['STORIES'] = df.groupby(['TAXCLASS'])['STORIES'].transform(lambda x: x.fillna(x.median()))

## Filling FULLVAL, AVLAND, AVTOT

In [50]:
# fill in missing value of FULL, AVLAND, AVTT
VAL=['FULLVAL', 'AVLAND', 'AVTOT']
df.loc[df['FULLVAL']==0, 'FULLVAL'] = np.nan
df.loc[df['AVLAND']==0, 'AVLAND'] = np.nan
df.loc[df['AVTOT']==0, 'AVTOT'] = np.nan
for i in VAL:
    df[i]=df.groupby(['TAXCLASS','ZIP'])[i].transform(lambda x: x.fillna(x.median()) if pd.notna(x).sum()>=5 else x)
    df[i]=df.groupby(['TAXCLASS'])[i].transform(lambda x: x.fillna(x.median()))   

# Filling LTFRONT & LTDEPTH

In [51]:
df.loc[:,'B'] = ['B'+str(i) for i in df['B']]
df.loc[:,'TAXCLASS'] = ['T'+str(i) for i in df['TAXCLASS']]


In [52]:
df['Group_Zip_B_TAX']=pd.Series(df['ZIP'])+pd.Series(df['B'])+pd.Series(df['TAXCLASS'])

In [57]:
df.loc[df['LTFRONT'] ==0,'LTFRONT'] = np.nan
df.loc[df['LTDEPTH'] ==0,'LTDEPTH'] = np.nan

In [58]:
df['LTDEPTH']=df.groupby(['Group_Zip_B_TAX'])['LTDEPTH'].transform(lambda x: x.fillna(x.median()) if pd.notna(x).sum()>=5 else x)
df['LTDEPTH']=df.groupby(['ZIP','B'])['LTDEPTH'].transform(lambda x: x.fillna(x.median()) if pd.notna(x).sum()>=5 else x)
df['LTDEPTH']=df.groupby(['ZIP'])['LTDEPTH'].transform(lambda x: x.fillna(x.median()) if pd.notna(x).sum()>=5 else x)
df['LTDEPTH']=df.groupby(['B'])['LTDEPTH'].transform(lambda x: x.fillna(x.median()))

df['LTFRONT']=df.groupby(['Group_Zip_B_TAX'])['LTFRONT'].transform(lambda x: x.fillna(x.median()) if pd.notna(x).sum()>=5 else x)
df['LTFRONT']=df.groupby(['ZIP','B'])['LTFRONT'].transform(lambda x: x.fillna(x.median()) if pd.notna(x).sum()>=5 else x)
df['LTFRONT']=df.groupby(['ZIP'])['LTFRONT'].transform(lambda x: x.fillna(x.median()) if pd.notna(x).sum()>=5 else x)
df['LTFRONT']=df.groupby(['B'])['LTFRONT'].transform(lambda x: x.fillna(x.median()))

## Filling BLDFRONT & BLDDEPTH

In [59]:
df.loc[df['BLDFRONT'] ==0,'BLDFRONT'] = np.nan
df.loc[df['BLDDEPTH'] ==0,'BLDDEPTH'] = np.nan

In [60]:
df['BLDDEPTH']=df.groupby(['Group_Zip_B_TAX'])['BLDDEPTH'].transform(lambda x: x.fillna(x.median()) if pd.notna(x).sum()>=5 else x)
df['BLDDEPTH']=df.groupby(['ZIP','B'])['BLDDEPTH'].transform(lambda x: x.fillna(x.median()) if pd.notna(x).sum()>=5 else x)
df['BLDDEPTH']=df.groupby(['ZIP'])['BLDDEPTH'].transform(lambda x: x.fillna(x.median()) if pd.notna(x).sum()>=5 else x)
df['BLDDEPTH']=df.groupby(['B'])['BLDDEPTH'].transform(lambda x: x.fillna(x.median()))

df['BLDFRONT']=df.groupby(['Group_Zip_B_TAX'])['BLDFRONT'].transform(lambda x: x.fillna(x.median()) if pd.notna(x).sum()>=5 else x)
df['BLDFRONT']=df.groupby(['ZIP','B'])['BLDFRONT'].transform(lambda x: x.fillna(x.median()) if pd.notna(x).sum()>=5 else x)
df['BLDFRONT']=df.groupby(['ZIP'])['BLDFRONT'].transform(lambda x: x.fillna(x.median()) if pd.notna(x).sum()>=5 else x)
df['BLDFRONT']=df.groupby(['B'])['BLDFRONT'].transform(lambda x: x.fillna(x.median()))

In [61]:
# check all the missing values
print(df.isnull().sum())

RECORD                   0
BBLE                     0
B                        0
BLOCK                    0
LOT                      0
EASEMENT           1066358
OWNER                31745
BLDGCL                   0
TAXCLASS                 0
LTFRONT                  0
LTDEPTH                  0
EXT                 716689
STORIES                  0
FULLVAL                  0
AVLAND                   0
AVTOT                    0
EXLAND                   0
EXTOT                    0
EXCD1               432506
STADDR                 676
ZIP                      0
EXMPTCL            1055415
BLDFRONT                 0
BLDDEPTH                 0
AVLAND2             788268
AVTOT2              788262
EXLAND2             983545
EXTOT2              940166
EXCD2               978046
PERIOD                   0
YEAR                     0
VALTYPE                  0
Group_Zip_B_TAX          0
dtype: int64


In [62]:
df[df['FULLVAL']==0].count()

RECORD             0
BBLE               0
B                  0
BLOCK              0
LOT                0
EASEMENT           0
OWNER              0
BLDGCL             0
TAXCLASS           0
LTFRONT            0
LTDEPTH            0
EXT                0
STORIES            0
FULLVAL            0
AVLAND             0
AVTOT              0
EXLAND             0
EXTOT              0
EXCD1              0
STADDR             0
ZIP                0
EXMPTCL            0
BLDFRONT           0
BLDDEPTH           0
AVLAND2            0
AVTOT2             0
EXLAND2            0
EXTOT2             0
EXCD2              0
PERIOD             0
YEAR               0
VALTYPE            0
Group_Zip_B_TAX    0
dtype: int64

In [63]:
df.to_csv('Filled Dataset.csv')

# Build Variables

In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


mydata = pd.read_csv('Filled Dataset.csv')

## Build Variables

# 0. only keep columns we need
mydata = mydata[['FULLVAL', 
         'AVLAND', 
         'AVTOT', 
         'LTFRONT', 
         'LTDEPTH', 
         'BLDFRONT', 
         'BLDDEPTH', 
         'STORIES', 
         'ZIP', 
         'TAXCLASS',
         'B']]

# 1. Create three sizes

mydata['lotarea']=mydata['LTFRONT']*mydata['LTDEPTH']
mydata['bldarea']=mydata['BLDFRONT']*mydata['BLDDEPTH']
mydata['bldvol']=mydata['bldarea']*mydata['STORIES']

# 2. Create nine variables: values nomalized by sizes

sizes=['lotarea','bldarea','bldvol']
values=['FULLVAL','AVLAND', 'AVTOT']

for n in values:
    for j in sizes:
        mydata[n+'/'+j]=mydata[n]/mydata[j]

# 3. Build new groups
        
# create variables
mydata['zip3'] = mydata['ZIP']//100
mydata['zip5'] = mydata['ZIP']//1

# 4 & 5. Calculate average of each group, and for each record, divided by the group average

normalized_values=['FULLVAL/lotarea',
 'FULLVAL/bldarea',
 'FULLVAL/bldvol',
 'AVLAND/lotarea',
 'AVLAND/bldarea',
 'AVLAND/bldvol',
 'AVTOT/lotarea',
 'AVTOT/bldarea',
 'AVTOT/bldvol']
groups=['zip3','zip5','TAXCLASS','B']
for i in normalized_values:
    mydata[i+'_by_all']=mydata[i]/mydata[i].mean()
    for j in groups:
        mydata[i+'_by_'+j]=mydata.groupby(j)[i].apply(lambda x: x/(x.mean()))

# 6. clean process data
mydata=mydata.drop(['FULLVAL',
 'AVLAND',
 'AVTOT',
 'LTFRONT',
 'LTDEPTH',
 'BLDFRONT',
 'BLDDEPTH',
 'STORIES',
 'ZIP',
 'TAXCLASS',
 'B',
 'lotarea',
 'bldarea',
 'bldvol',
 'FULLVAL/lotarea',
 'FULLVAL/bldarea',
 'FULLVAL/bldvol',
 'AVLAND/lotarea',
 'AVLAND/bldarea',
 'AVLAND/bldvol',
 'AVTOT/lotarea',
 'AVTOT/bldarea',
 'AVTOT/bldvol',
 'zip3',
 'zip5'], axis=1)

# Look at the current dataset
mydata.head()


mydata.shape

mydata.to_csv('02 Variable Created.csv', index=False)

# Dimensionality Reduction

In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Dimensionality Reduction

mydata = pd.read_csv('02 Variable Created.csv')

mydata.head()

# Z scale before PCA
columns = mydata.columns
for column in columns:
    mydata[column] = (mydata[column]-mydata[column].mean())/mydata[column].std()

mydata.head()

# Performing PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=15)
PCAdf = pca.fit_transform(mydata.iloc[:, 1:])
PCAdf = pd.DataFrame(data=PCAdf)

PCAscore = pca.explained_variance_ratio_
PCAscore
# Keep the first five PCs, which explain roughly 90% of the variance

PCAdf.head()

# Z-scale again after PCA
from sklearn.preprocessing import StandardScaler
PCAdf = pd.DataFrame(StandardScaler().fit_transform(PCAdf)).iloc[:,0:5]

PCAdf.head()

PCAdf.to_csv('03 PCA.csv', index=False)

### Fraud score one calculation - Heuristic use of Z scores

PCAdf['Score1'] = (PCAdf[0]**2 + PCAdf[1]**2 + PCAdf[2]**2 + PCAdf[3]**2 + PCAdf[4]**2)**0.5

PCAdf.head()

,0,1,2,3,4,Score1
0,-0.026545,-0.104346,-0.104095,0.016987,0.092443,0.176813
1,4.779884,18.902035,10.416541,4.686295,1.249922,22.631001
2,0.019498,0.101076,-0.068948,-0.081352,0.084040,0.170385
3,0.051831,-0.087467,-0.130390,-0.028574,-0.091078,0.190919
4,9.919771,-4.969900,-4.402207,-0.629945,-26.326771,28.913265


# Fraud score 2nd calculation - Autoencoder

In [69]:
# pip install --upgrade tensorflow

from scipy import stats
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.preprocessing import StandardScaler
from pylab import rcParams
# from keras.models import Model, load_model
# from keras.layers import Input, Dense
# from keras.callbacks import ModelCheckpoint, TensorBoard
# from keras import regularizers
from tensorflow.keras.models import Model, load_model,Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras import regularizers

put_dim = PCAdf.shape[1]-1
put_dim

ATC = PCAdf.iloc[:,:-1]
ATC

input_dim = ATC.shape[1] #num of columns
encoding_dim = 3
# hidden_dim = int(encoding_dim / 2) 
learning_rate = 10e-5

input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim, activation="tanh", activity_regularizer=regularizers.l1(learning_rate))(input_layer)
# encoder = Dense(hidden_dim, activation="relu")(encoder)
# decoder = Dense(hidden_dim, activation='tanh')(encoder)
decoder = Dense(input_dim, activation='relu')(encoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

nb_epoch = 1
autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='adam')

cp = ModelCheckpoint(filepath="autoencoder_fraud.h5",
                               save_best_only=True,
                               verbose=0)

tb = TensorBoard(log_dir='./logs',
                histogram_freq=0,
                write_graph=True,
                write_images=True)

history = autoencoder.fit(ATC, ATC,
                    epochs=nb_epoch,
#                     batch_size=batch_size,
                    shuffle=True,
#                     validation_data=(test_x, test_x),
                    verbose=1,
                    callbacks=[cp, tb]).history

predictions = autoencoder.predict(ATC)
predictions

autoencoded_data = pd.DataFrame(predictions)
autoencoded_data.columns = ['ACT-0','ACT-1','ACT-2','ACT-3','ACT-4']
autoencoded_data 




Train on 1070994 samples
1070994/1070994 [==============================] - 30s 28us/sample - loss: 0.9769 - accuracy: 0.3133


,ACT-0,ACT-1,ACT-2,ACT-3,ACT-4
0,0.005066,0.000000,0.000000,0.000000,0.0
1,1.948611,4.406554,3.404198,0.000000,0.0
2,0.000000,0.084751,0.000000,0.000000,0.0
3,0.036684,0.000000,0.000000,0.004832,0.0
4,5.219816,0.000000,0.000000,2.736728,0.0
...,...,...,...,...,...
1070989,0.000000,0.000000,0.000000,0.000000,0.0
1070990,0.000000,0.000000,0.000000,0.000000,0.0
1070991,0.000000,0.000000,0.000000,0.000000,0.0
1070992,0.000000,0.000000,0.000000,0.000000,0.0


In [70]:
## Calculate Diff:
for i in range(5):
    ATC['diff'+str(i)] = ATC.iloc[:,i]-autoencoded_data.iloc[:,i]

ATC

score = (ATC['diff0']**2 + ATC['diff1']**2 +ATC['diff2']**2 \
        +ATC['diff3']**2 +ATC['diff4']**2 )**(1/2)

score

PCAdf['Score2'] = score

PCAdf.head()


,0,1,2,3,4,Score1,Score2
0,-0.026545,-0.104346,-0.104095,0.016987,0.092443,0.176813,0.177644
1,4.779884,18.902035,10.416541,4.686295,1.249922,22.631001,17.053788
2,0.019498,0.101076,-0.068948,-0.081352,0.084040,0.170385,0.138135
3,0.051831,-0.087467,-0.130390,-0.028574,-0.091078,0.190919,0.185182
4,9.919771,-4.969900,-4.402207,-0.629945,-26.326771,28.913265,27.759724


# Quantile binning & rank ordering

In [71]:
PCAdf['Rank_score1']=PCAdf['Score1'].rank(method='dense',ascending=False)
PCAdf['Rank_score2']=PCAdf['Score2'].rank(method='dense',ascending=False)

PCAdf.head(10)

PCAdf['Rank']=0.5*PCAdf['Rank_score1']+0.5*PCAdf['Rank_score2']
PCAdf = PCAdf.sort_values(by=['Rank'], ascending=True)

Top10=PCAdf.head(10)

Top10

rownumber=Top10.index

filleddata = pd.read_csv('Filled Dataset.csv')

filleddata.iloc[rownumber,:]

mydata.iloc[rownumber,:]

,FULLVAL/lotarea_by_all,FULLVAL/lotarea_by_zip3,FULLVAL/lotarea_by_zip5,FULLVAL/lotarea_by_TAXCLASS,FULLVAL/lotarea_by_B,FULLVAL/bldarea_by_all,FULLVAL/bldarea_by_zip3,FULLVAL/bldarea_by_zip5,FULLVAL/bldarea_by_TAXCLASS,FULLVAL/bldarea_by_B,...,AVTOT/bldarea_by_all,AVTOT/bldarea_by_zip3,AVTOT/bldarea_by_zip5,AVTOT/bldarea_by_TAXCLASS,AVTOT/bldarea_by_B,AVTOT/bldvol_by_all,AVTOT/bldvol_by_zip3,AVTOT/bldvol_by_zip5,AVTOT/bldvol_by_TAXCLASS,AVTOT/bldvol_by_B
632815,-0.014102,0.004798,-0.016538,0.253599,0.023832,342.174335,340.441166,365.989357,840.662786,323.710460,...,342.490756,341.517208,383.059990,812.395565,317.933273,606.688005,449.795922,406.564452,1021.823582,526.138932
565391,325.035199,209.692428,419.315256,269.555024,208.803092,336.824876,361.723618,361.134985,164.578178,339.003567,...,337.137152,487.085507,460.545561,150.858037,456.137182,597.204915,695.239828,475.757843,92.626965,680.066066
1067359,793.939859,936.830649,543.428102,668.978099,932.811342,-0.009070,-0.005522,-0.070432,-0.011224,-0.005175,...,-0.010021,-0.006048,-0.033236,0.000189,-0.005663,-0.006816,-0.005753,-0.029357,0.000278,-0.005626
917941,0.522897,0.667091,1.384819,0.415924,0.520662,18.134000,19.445363,28.075544,8.853164,17.148898,...,505.271087,632.876034,304.740440,226.108639,469.042013,298.333521,346.833703,261.612668,46.266804,258.723240
585117,-0.173408,-0.130036,-0.171206,-0.161566,-0.123557,402.142904,327.750407,218.706116,196.495159,380.444318,...,402.505783,156.115487,117.072551,180.114646,373.645133,35.636529,30.618019,45.664567,5.517923,30.903308
85885,-0.089549,-0.103438,-0.101787,-0.092017,-0.101848,128.078572,148.089650,251.653086,62.576446,142.028820,...,128.229132,58.909401,167.383913,57.358444,56.934325,227.156337,193.360935,273.612591,35.225982,197.921122
585438,0.026634,0.012272,0.228471,0.004341,0.061521,433.517189,353.322387,235.774213,211.825880,410.126270,...,433.904373,168.294595,126.208683,194.167507,402.792398,76.848777,66.029277,98.507522,11.910661,66.643997
585119,-0.157718,-0.113919,-0.167757,-0.148554,-0.109040,251.251986,249.977113,304.758183,122.763866,237.692473,...,251.497969,250.782989,316.507033,112.529069,233.464803,22.262167,16.501229,24.521464,3.443326,19.304598
565397,4.467947,2.862755,5.796620,3.687810,2.850621,179.881851,193.177687,192.851091,87.889585,181.044098,...,180.072543,260.166957,245.975545,80.561660,243.636532,318.989004,371.353923,254.102960,49.470829,363.249044
920627,-0.002376,0.117722,0.212315,-0.017088,0.034682,221.863416,214.925312,279.188522,112.218879,209.889126,...,19.663010,23.988970,94.807426,99.060968,18.252754,17.417064,16.480356,81.029573,30.240834,15.102757
